In [ ]:
from google.cloud import geminidataanalytics
from google.adk.agents import Agent
# Removed VertexAiCodeExecutor and other executor imports
from google.adk.tools import agent_tool
import vertexai
# from vertexai import agent_engines # Kept for potential vertexai.init
# from vertexai.preview import reasoning_engines # Kept for potential vertexai.init
import json # Import json for potentially parsing the data

def get_insights(question: str):
    """Queries the Conversational Analytics API using a question as input.

  Use this tool to generate the data for data insights.

  Args:
      question: The question to post to the API.

  Returns:
      A dictionary containing the status of the operation and the insights from
      the API, categorized by type (e.g., text_insights, data_insights) to make
      the output easier for an LLM to understand and process.
  """

    data_chat_client = geminidataanalytics.DataChatServiceClient()
    lookml_model = "gaming"
    explore = "events"

    # User Credentials
    looker_client_id = "9cR2K4JdGYjZCBCm6HGs"
    looker_client_secret = "8YP9CWFVhdzxF2dvPsyJhQdR"
    # Required only for Looker PUBLIC instance
    looker_instance_uri = "https://3417a175-fe20-4370-974f-2f2b535340ab.looker.app"

    credentials = geminidataanalytics.Credentials(
        oauth=geminidataanalytics.OAuthCredentials(
            secret=geminidataanalytics.OAuthCredentials.SecretBased(
                client_id=looker_client_id, client_secret=looker_client_secret
            ),
        )
    )

    looker_explore_reference = geminidataanalytics.LookerExploreReference(
        looker_instance_uri=looker_instance_uri, lookml_model=lookml_model, explore=explore
    )

    # Connect to your Looker datasource
    datasource_references = geminidataanalytics.DatasourceReferences(
        looker=geminidataanalytics.LookerExploreReferences(
            explore_references=[looker_explore_reference],
            credentials=credentials # Note: uncomment this only in case of stateless chat via inline context
        ),
    )

    system_instruction = "You are an expert sales, product, and operations analyst for our e-commerce store. Your primary function is to answer questions by querying the 'Order Items' Explore. Always be concise and data-driven. Never generate a chart."

    # Context set-up for 'Chat using Inline Context'
    inline_context = geminidataanalytics.Context(
        system_instruction=system_instruction,
        datasource_references=datasource_references,
        options=geminidataanalytics.ConversationOptions(
            analysis=geminidataanalytics.AnalysisOptions(
                python=geminidataanalytics.AnalysisOptions.Python(
                    enabled=False
                )  # if wanting to use advanced analysis with python
            )
        ),
    )

    billing_project = "update"

    messages = [geminidataanalytics.Message()]
    messages[0].user_message.text = question

    request = geminidataanalytics.ChatRequest(
        inline_context=inline_context,
        parent=f"projects/{billing_project}/locations/global",
        messages=messages,
    )

    # Make the request
    stream = data_chat_client.chat(request=request)

    # Categorize insights from the stream for a more descriptive output
    text_insights = []
    schema_insights = []
    data_insights = []
    # chart_insights = [] # No longer needed

    for item in stream:
        if item._pb.WhichOneof("kind") == "system_message":
            message_dict = geminidataanalytics.SystemMessage.to_dict(
                item.system_message
            )
            if "text" in message_dict:
                text_insights.append(message_dict["text"])
            elif "schema" in message_dict:
                schema_insights.append(message_dict["schema"])
            elif "data" in message_dict:
                data_insights.append(message_dict["data"])
            # elif "chart" in message_dict: # chart_insights removed
            #     chart_insights.append(message_dict["chart"])

    # Build a descriptive response dictionary that is easier for the LLM to parse
    response = {"status": "success"}
    if text_insights:
        response["text_insights"] = text_insights
    if schema_insights:
        response["schema_insights"] = schema_insights
    if data_insights:
        response["data_insights"] = data_insights
    # chart_insights removed from response
    print(response)
    return response

# Agent to get data insights
data_agent = Agent(
    model="gemini-2.5-flash-lite",
    name="DataAgent",
    description="Use this agent to get data insights about orders, sales, and e-commerce from the Cymbal Pets Superstore.",
    instruction = """You are an agent that retrieves raw data. The tool 'get_insights' queries a governed semantic layer.
    Your task is to call the 'get_insights' tool with the user's question.
    From the tool's dictionary output, find the 'data_insights' list. Within that list, find the dictionary that contains a 'result' key.
    Extract the list of records from the 'data' key which is inside 'result'.
    Return this list of records as a JSON string. Do not add any other text or summarization.
    """,
    tools=[get_insights],
)

# visualization_agent has been removed.

# Root agent to orchestrate the sub-agents
root_agent = Agent(
    model="gemini-2.5-flash-lite",
    name="RootAgent",
    instruction="""You are a helpful data analysis assistant. Your goal is to answer user questions by retrieving data.

    Your workflow is as follows:
    1. When the user asks a question about data, use the `DataAgent` tool to retrieve the necessary data. The `DataAgent` will return a JSON string of the raw data.
    2. Briefly summarize the findings from the JSON data provided by DataAgent. Present key results clearly. Use a markdown table if the data is tabular. Do not offer to create any visualizations or charts.
    """,
    tools=[
        agent_tool.AgentTool(agent=data_agent),
        # agent_tool.AgentTool(agent=visualization_agent), # Removed visualization tool
    ],
)

PROJECT_ID = "aragosalooker"
LOCATION = "us-central1"
STAGING_BUCKET = "ca_api"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

app = reasoning_engines.AdkApp(
    agent=root_agent,
    
    enable_tracing=True,
)

remote_app = agent_engines.create(
    agent_engine=app,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]",
        "google-cloud-geminidataanalytics",
    ],
)